<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,sklearn,matplotlib

Ethen 2018-11-13 12:28:19 

CPython 3.6.4
IPython 6.4.0

numpy 1.14.1
pandas 0.23.0
sklearn 0.19.1
matplotlib 2.2.2


In [2]:
# https://www.kaggle.com/girianantharaman/dual-embeddings-space-model-demo

DATA_DIR = os.path.join('data', 'all-the-news')
data_path = os.path.join(DATA_DIR, 'articles1.csv')

df = pd.read_csv(data_path, sep=',')
print('dimension: ', df.shape)
df.head()

dimension:  (50000, 10)


,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [31]:
content = df.loc[0, 'content']
content

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [32]:
def preprocess_text(text, nlp_parser):
    """
    Generator function using spaCy to parse reviews:
    - lemmatize the text
    - remove punctuation, whitespace and number
    - remove some pos tags
    """
    parsed_text = nlp_parser(text)
    cleaned_text = ' '.join(token.lemma_ for token in parsed_text if is_valid_word(token))
    return cleaned_text


def is_valid_word(token):
    """
    Returns False if the spacy token is either
    a punctuation, whitespace, number, or is one
    of the un-wanted POS tag

    http://universaldependencies.org/u/pos/
    """
    pos_flag = token.pos_ not in {'ADP', 'CCONJ', 'PRON'}
    word_flag = not (token.is_punct or token.is_space or token.like_num or token.is_stop)
    word_len_flag = len(token) >= 2
    valid_flag = pos_flag and word_flag and word_len_flag
    return valid_flag


nlp_parser = spacy.load('en_core_web_sm')
preprocess_text(content, nlp_parser)

"washington congressional republicans new fear come health care lawsuit obama administration win the incoming trump administration choose longer defend executive branch suit challenge administration ’s authority spend billion dollar health insurance subsidy americans hand house republicans big victory issue sudden loss dispute subsidy conceivably because health care program implode leave million people access health insurance republicans prepare replacement that lead chaos insurance market spur political backlash republicans gain control government to stave outcome republicans find awkward position appropriate huge sum temporarily prop obama health care law anger conservative voter demand end law year twist donald j. trump ’s administration worry preserve executive branch prerogative choose fight republican ally house central question dispute eager avoid ugly political pileup republicans capitol hill trump transition team game handle lawsuit election limbo late february united states c

In [33]:
from nltk import word_tokenize

??word_tokenize

In [3]:
CONTENT_INDEX = 9
CONTENT_PATH = './inputs/contents.txt'
TOKENS_PATH = './inputs/tokens.txt'
CENTROIDS_PATH = './inputs/centroids/'
BM25_PATH = './inputs/bm25/'

# if not os.path.isdir(CONTENT_PATH):
#     os.makedirs(CONTENT_PATH)

# if not os.path.isfile(TOKENS_PATH):
#     os.makedirs(TOKENS_PATH)
    
if not os.path.isdir(CENTROIDS_PATH):
    os.makedirs(CENTROIDS_PATH)

if not os.path.isdir(BM25_PATH):
    os.makedirs(BM25_PATH)

In [24]:
import csv
import sys
import glob
import nltk
import time
from tqdm import tqdm


def preprocess_content():
    csv.field_size_limit(sys.maxsize)

    nlp_parser = spacy.load('en_core_web_sm')

    article_files = os.path.join(DATA_DIR, '*.csv')
    for fname in glob.iglob(article_files):

        with open(fname, 'r') as fin, open(CONTENT_PATH, 'w') as fout:
            
            # skip the header
            reader = csv.reader(fin)
            next(reader)
            for line in tqdm(reader):
                try:
                    content = line[CONTENT_INDEX]
                    cleaned_content = preprocess_text(content, nlp_parser)
                    fout.write(cleaned_content + '\n')
    #                 for sent in nltk.sent_tokenize(content):
    #                     fout.write(sent)

                except IndexError:
                    print(line)
                
start = time.time()             
preprocess_content()
elapsed = time.time() - start
print('elapsed: ', elapsed)

487it [01:33,  4.32it/s]

KeyboardInterrupt: 

In [18]:
import spacy

nlp_parser = spacy.load('en_core_web_sm')

preprocess_text(content, nlp_parser)

"washington congressional republicans new fear come health care lawsuit obama administration win the incoming trump administration choose longer defend executive branch suit challenge administration ’s authority spend billion dollar health insurance subsidy americans hand house republicans big victory issue sudden loss dispute subsidy conceivably because health care program implode leave million people access health insurance republicans prepare replacement that lead chaos insurance market spur political backlash republicans gain control government to stave outcome republicans find awkward position appropriate huge sum temporarily prop obama health care law anger conservative voter demand end law year twist donald j. trump ’s administration worry preserve executive branch prerogative choose fight republican ally house central question dispute eager avoid ugly political pileup republicans capitol hill trump transition team game handle lawsuit election limbo late february united states c

In [8]:
import csv
import glob
article_files = os.path.join(DATA_DIR, '*.csv')
fname = glob.glob(article_files)[0]
with open(fname, 'r') as fin:
    
    # skip the header
    reader = csv.reader(fin)
    next(reader)
    for line in reader:
        content = line[CONTENT_INDEX]
        break

In [45]:
import logging
from gensim.models import Word2Vec


class MySentences:
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename) as f:
            for line in f:
                yield nltk.word_tokenize(line)

In [46]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = MySentences(TOKENS_PATH)
model = Word2Vec(sentences, min_count=1)

2018-11-12 14:26:24,576 : INFO : collecting all words and their counts
2018-11-12 14:27:51,504 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-11-12 14:28:38,724 : INFO : collected 854955 word types from a corpus of 39042305 raw words and 2 sentences
2018-11-12 14:28:38,967 : INFO : Loading a fresh vocabulary
2018-11-12 14:28:41,047 : INFO : min_count=1 retains 854955 unique words (100% of original 854955, drops 0)
2018-11-12 14:28:41,048 : INFO : min_count=1 leaves 39042305 word corpus (100% of original 39042305, drops 0)
2018-11-12 14:28:42,986 : INFO : deleting the raw counts dictionary of 854955 items
2018-11-12 14:28:43,003 : INFO : sample=0.001 downsamples 42 most-common words
2018-11-12 14:28:43,004 : INFO : downsampling leaves estimated 29547512 word corpus (75.7% of prior 39042305)
2018-11-12 14:28:45,365 : INFO : estimated required memory for 854955 words and 100 dimensions: 1111441500 bytes
2018-11-12 14:28:45,366 : INFO : resetting layer weig

In [ ]:
def clean_up(text):  # clean up your text and generate list of words for each document. 
    removal = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
    text_out = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False and token.is_alpha and len(token)>2 and token.pos_ not in removal:
            lemma = token.lemma_
            text_out.append(lemma)

    return text_out

In [62]:
# model.trainables.syn1neg

array([[-1.8023539e-01, -2.2743858e-02,  1.8745484e-02, ...,
        -5.4894388e-02, -1.0650578e-01,  9.9624721e-03],
       [-1.5879828e-01, -2.2303106e-02,  1.7812191e-02, ...,
        -4.9874853e-02, -9.3539819e-02,  9.1556050e-03],
       [-1.0424251e-01, -1.4368367e-02,  1.1174218e-02, ...,
        -3.1296801e-02, -6.3423023e-02,  6.2797209e-03],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-4.6039884e-05, -6.1764463e-06,  5.2200508e-06, ...,
        -1.5610445e-05, -2.8076762e-05,  4.9417058e-06],
       [-7.3054136e-05, -1.1287094e-05,  2.5387400e-05, ...,
        -2.1319236e-05, -3.4774755e-05,  7.6615042e-06]], dtype=float32)

In [49]:
model.wv.most_similar(positive=['texas', 'senate'], negative='alabama')

2018-11-12 14:49:27,510 : INFO : precomputing L2-norms of word weight vectors


[('1373.', 0.46903932094573975),
 ('believes.', 0.43915441632270813),
 ('Santino.Bannan', 0.4375481903553009),
 ('process.Woodrow', 0.4163869321346283),
 ('matriarchal', 0.41474297642707825),
 ('throws.According', 0.41017037630081177),
 ('July.Page', 0.4094330072402954),
 ('opinion.After', 0.40884116291999817),
 ('Villanova.Worst', 0.4088144600391388),
 ('Blues.It', 0.4059373736381531)]

In [ ]:
model.save('./model/w2v-lc.model')
model.wv.save_word2vec_format('./model/w2v-lc.model.bin', binary=True)
vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
with open('./model/w2v-lc-vocab.json', 'w') as f:
    f.write(json.dumps(vocab))

In [ ]:
def get_embedding(x, out=False):
    if x in model.wv.vocab:
        if out == True:
            return model.syn1neg[model.wv.vocab[x].index]
        else:
            return model[x]
    else:
        return np.zeros(100)